# BST281 Final Project Pipeline

Group 2  
Dongyuan Song, Siquan Wang, Xutao Wang, Linying Zhang

## Set Up
Import packages; set working direcotries.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
rcParams['font.sans-serif'] = 'Arial'
import warnings
warnings.filterwarnings("ignore")
import urllib3
urllib3.disable_warnings()
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from IPython.display import FileLinks
from IPython.display import FileLink
import IPython

In [22]:
current_path = os.getcwd()
print(current_path)

C:\Users\songdongyuan\group02_final_project_packet


Set working directory. Default is this package folder.

In [23]:
os.chdir(current_path)

Enable using R in Jupyter notebook.

In [24]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Function for displaying Bash code

In [24]:
def highlight_source_bash(filename):
    """For use inside an IPython notebook: given a filename, print the source code. Bash version."""

    from pygments import highlight
    from pygments.lexers import BashLexer
    from pygments.formatters import HtmlFormatter
    from IPython.core.display import HTML

    with open (filename, "r") as myfile:
        data = myfile.read()

    return HTML(highlight(data, BashLexer(), HtmlFormatter(full=True)))

Function for displaying R code

In [33]:
def highlight_source_r(filename):
    """For use inside an IPython notebook: given a filename, print the source code. R version."""

    from pygments import highlight
    from pygments.lexers import SLexer
    from pygments.formatters import HtmlFormatter
    from IPython.core.display import HTML

    with open (filename, "r") as myfile:
        data = myfile.read()

    return HTML(highlight(data, SLexer(), HtmlFormatter(full=True)))

## RNA-seq analysis

### Check raw data

In [25]:
expr_df = pd.read_csv("expressionFile_counts_MM.csv")

In [26]:
expr_df = expr_df.set_index(expr_df.columns[0])
expr_df.head()

,..NM89_RPMI_salmon.quant.sf,..NM90_RPMI_HS5_salmon.quant.sf,..NM91_MM1S_salmon.quant.sf,..NM92_MM1S_HS5_salmon.quant.sf,..NM95_KMS11_salmon.quant.sf,..NM96_KMS11_HS5_salmon.quant.sf
Name,,,,,,
5_8S_rRNA,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5S_rRNA,6.45945,21.44398,10.03000,10.11391,0.00000,1.01000
7SK,3.03000,3.26734,0.78000,0.00000,0.00000,2.78045
A1BG,980.97371,1196.18930,38.39037,79.96080,4.68050,20.19474
A1BG-AS1,944.94700,1099.25405,3.76547,21.01000,1.84924,2.56537


### Quality Control
Filter out none or low expressed genes.

In [27]:
print(expr_df.shape)

(58671, 6)


Here we only keep genes which counts are larger than 1 in each samples.

In [28]:
mask_low_vals = (expr_df > 0).sum(axis=1) == 6
expr_df = expr_df.loc[mask_low_vals, :]
print(expr_df.shape)

(22366, 6)


Save the result in working directory.

In [29]:
expr_df.to_csv('filtered.tsv',sep='\t')

### Normalization and Differential Expression Analysis

This step was finished in R. Use Bioconductor Package *edgeR*, *limma* and *DEseq2*.

In [38]:
!Rscript RNA_seq.R


out of 22366 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 1, 0.0045% 
LFC < 0 (down)   : 156, 0.7% 
outliers [1]     : 604, 2.7% 
low counts [2]   : 0, 0% 
(mean count < 1)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



Warning messages:
1: 程辑包'GenomicRanges'是用R版本3.4.4 来建造的 
2: 程辑包'matrixStats'是用R版本3.4.4 来建造的 
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


The codes are showed below.

In [34]:
highlight_source_r("RNA_seq.R")

Since here almost all differential expressed genes were up regulated, we only do functional analysis on them.

See the list of DE genes.

In [43]:
rna_de = pd.read_csv("DEgene_list.tsv", header=None)

In [44]:
rna_de.head()

,0
0,ABI3BP
1,ABLIM3
2,ACKR3
3,ADAM12
4,ADAMTS1


### Enrichment analysis

Use **David** <https://david.ncifcrf.gov/> to perform online enrichment analysis. See the result of KEGG and Biological Process below.

In [50]:
rna_KEGG = pd.read_csv("RNA_seq_David_KEGG.txt", sep='\t')

In [51]:
rna_KEGG.head()

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR
0,KEGG_PATHWAY,hsa04512:ECM-receptor interaction,12,7.741935,7.803741e-10,"LAMA1, COL4A2, LAMA4, TNC, COL1A2, COL6A2, COL...",71,87,6910,13.423992,1.186169e-07,1.186169e-07,9.340311e-07
1,KEGG_PATHWAY,hsa04151:PI3K-Akt signaling pathway,19,12.258065,5.137308e-09,"CSF3, EGFR, FGF5, COL4A2, IL6, OSMR, TNC, LPAR...",71,345,6910,5.359869,7.808704e-07,3.904353e-07,6.148852e-06
2,KEGG_PATHWAY,hsa04510:Focal adhesion,15,9.677419,1.249131e-08,"EGFR, COL4A2, TNC, COL5A2, COL5A1, MYL9, LAMA1...",71,206,6910,7.086695,1.898677e-06,6.328926e-07,1.495086e-05
3,KEGG_PATHWAY,hsa05146:Amoebiasis,10,6.451613,9.977432e-07,"LAMA1, COL4A2, LAMA4, IL6, COL1A2, CXCL8, IL1B...",71,106,6910,9.181504,1.516455e-04,3.791354e-05,1.194194e-03
4,KEGG_PATHWAY,hsa04974:Protein digestion and absorption,9,5.806452,2.487070e-06,"COL4A2, COL1A2, COL6A2, MME, COL6A1, COL1A1, C...",71,88,6910,9.953585,3.779637e-04,7.560416e-05,2.976738e-03


In [52]:
rna_BP = pd.read_csv("RNA_seq_David_BP.txt", sep='\t')
rna_BP.head()

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR
0,GOTERM_BP_DIRECT,GO:0030198~extracellular matrix organization,23,14.838710,9.133737e-19,"MATN3, COL4A2, TNC, FBN1, CCDC80, DCN, COL5A2,...",143,196,16792,13.779649,1.064080e-15,1.064080e-15,1.471183e-15
1,GOTERM_BP_DIRECT,GO:0030574~collagen catabolic process,11,7.096774,1.394084e-10,"COL4A2, MRC2, COL1A2, COL6A2, COL6A1, COL1A1, ...",143,64,16792,20.182692,1.624107e-07,8.120537e-08,2.245468e-07
2,GOTERM_BP_DIRECT,GO:0045766~positive regulation of angiogenesis,11,7.096774,4.758890e-08,"SASH1, C3, F3, CCBE1, SERPINE1, CXCL8, IL1B, T...",143,115,16792,11.232107,5.543953e-05,1.848018e-05,7.665201e-05
3,GOTERM_BP_DIRECT,GO:0007155~cell adhesion,18,11.612903,3.442427e-07,"TNC, ACKR3, COL5A1, LAMA1, CDH13, LGALS3BP, LA...",143,459,16792,4.604964,4.009625e-04,1.002557e-04,5.544747e-04
4,GOTERM_BP_DIRECT,GO:0030199~collagen fibril organization,7,4.516129,8.528437e-07,"COL1A2, COL1A1, LOX, GREM1, LOXL2, COL5A2, COL5A1",143,39,16792,21.076564,9.930699e-04,1.986929e-04,1.373677e-03


## Mint-ChIP analysis

### Quality Control

The input Mint-ChIP files are BAM file already. Use **fastqc** to do quality control. Computation was done on clusters

In [33]:
%%bash
sbatch fastqc.sh

-bash: line 1: sbatch: command not found


The codes are showed below.

In [25]:
highlight_source_bash("fastqc.sh")

Show the fastqc reports.

In [35]:
FileLinks(os.path.join('./fastqc_output'), included_suffixes=['.html'])

./fastqc_output/
  Alignment_Post_Processing_15005_fastqc.html
  Alignment_Post_Processing_15009_fastqc.html
  Alignment_Post_Processing_15022_fastqc.html
  Alignment_Post_Processing_15175_fastqc.html
  Alignment_Post_Processing_15180_fastqc.html
  Alignment_Post_Processing_15193_fastqc.html
  Alignment_Post_Processing_15223_fastqc.html
  Alignment_Post_Processing_15280_fastqc.html

The reports show that the quality is fine. Use the BAM file for next step.

### Peak Calling

Use MACS2 to do peak calling. Notice some parameters: file type is BAMPE, q = 0.01. Cimputation was done on clusters.

In [36]:
%%bash
sbatch MACS2.sh

-bash: line 1: sbatch: command not found


The codes are showed below.

In [26]:
highlight_source_bash("MACS2.sh")

The peak calling results are showed below.

In [38]:
FileLinks(os.path.join('./macs2_output'), included_suffixes=['.xls'])

./macs2_output/
  15005_peaks.xls
  15009_peaks.xls
  15022_peaks.xls
  15175_peaks.xls
  15180_peaks.xls
  15193_peaks.xls

### Differential Binding Analysis

### Motif Analysis

Use Homer to do Motif Analysis. Computation was done on clusters.

In [39]:
%%bash
sbatch homer.sh

-bash: line 1: sbatch: command not found


The codes are showed below.

In [27]:
highlight_source_bash("homer.sh")

The motif analysis results are showed below.

Result of H3K4me3 increasing binding sites

In [53]:
FileLink("./homer_output/H3K4me3_increase/knownResults.html")

C:\Users\songdongyuan\group02_final_project_packet\homer_output\H3K4me3_increase\knownResults.html

Result of H3K4me3 decreasing binding sites

In [55]:
FileLink("./homer_output/H3K4me3_decrease/knownResults.html")

C:\Users\songdongyuan\group02_final_project_packet\homer_output\H3K4me3_decrease\knownResults.html

Result of H3K27ac increasing binding sites

In [58]:
FileLink("./homer_output/H3K27ac_increase/knownResults.html")

C:\Users\songdongyuan\group02_final_project_packet\homer_output\H3K27ac_increase\knownResults.html

Result of H3K27ac
decreasing binding sites

In [59]:
FileLink("./homer_output/H3K27ac_decrease/knownResults.html")

C:\Users\songdongyuan\group02_final_project_packet\homer_output\H3K27ac_decrease\knownResults.html